In [ ]:
import ee
ee.Initialize()

In [8]:
def addNDVI(image):
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def addNDVI75(image):
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)


def select_ls(year):
    if year >=2013:
        ls = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
    elif year >= 2003:
        ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")
    elif year >= 1999:
        ls = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA").merge(ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA"))
    else:
        ls = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")
    return ls


def get_ndvi(image, shp, year):
    image = image.filterBounds(shp).filterDate(str(year) + '-05-1', str(year) + '-11-1').filter(ee.Filter.lt('CLOUD_COVER', 30))
    if year >= 2013:
        image = image.map(addNDVI).select('NDVI').median()
    if year < 2013:
        image = image.map(addNDVI75).select('NDVI').median()
    return image


#export
def export_to_driver(image, year):
    taskParams = {
    'driveFolder': 'NE_tif_LandTrendr',
    'scale': 30,
    'maxPixels': 1e13
    }
    task = ee.batch.Export.image(image, str(year), taskParams)
    task.start()

In [9]:
roi = ee.Geometry.Polygon([[122.3333493386204,51.28079736294542],
[123.35782443627664,51.28079736294542],
[123.35782443627664,51.964558045579516],
[122.3333493386204,51.964558045579516],
[122.3333493386204,51.28079736294542]])

for year in range(1988, 2019):
    NDVI = get_ndvi(select_ls(year), roi, year)
    export_to_driver(NDVI, year)